In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    # plt.plot(history.history['val_'+string], '')
    plt.xlabel("Epochs")
    plt.ylabel(string)
    # plt.legend([string, 'val_'+string])
    plt.show()

In [3]:
df = pd.read_csv('Datasets/Raw_Dataset/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv')
df.head()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,16/02/2018 08:27:23,112640768,3,0,0,0,0,0,...,0,0,0,0,0,5.63e+07,138.593,56300000,56300000,Benign
1,0,0,16/02/2018 08:30:12,112641773,3,0,0,0,0,0,...,0,0,0,0,0,5.63e+07,263.751,56300000,56300000,Benign
2,35605,6,16/02/2018 08:26:55,20784143,23,44,2416,1344,240,64,...,20,2624734,0,2624734,2624734,9.05821e+06,0,9058214,9058214,Benign
3,0,0,16/02/2018 08:33:01,112640836,3,0,0,0,0,0,...,0,0,0,0,0,5.63e+07,82.0244,56300000,56300000,Benign
4,23,6,16/02/2018 08:27:59,20,1,1,0,0,0,0,...,20,0,0,0,0,0,0,0,0,Benign


In [4]:
features = ['Timestamp', 'Fwd Pkt Len Std', 'Fwd Pkt Len Mean',
       'Fwd Pkt Len Max', 'Fwd Seg Size Avg', 'Pkt Len Std', 'Flow IAT Std',
       'Bwd Pkt Len Std', 'Bwd Seg Size Avg', 'Pkt Size Avg',
       'Subflow Fwd Byts']

In [5]:
def targetify(s):
    if s == 'Benign':
        return 0
    else:
        return 1

In [6]:
df['Target']=df['Label'].apply(targetify)

In [7]:
X = df[features]
X[features] = X[features].apply(pd.to_numeric, errors='coerce', axis=1)

C:\Users\young\AppData\Roaming\Python\Python36\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [8]:
y = df['Target']

In [9]:
X = X.fillna(0)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print (X_train.shape, y_train.shape)
print( X_test.shape, y_test.shape)

(838860, 11) (838860,)
(209715, 11) (209715,)


In [11]:
# # selecting 1% of random rows for better running time

# X_train2 = X_train.sample(frac=0.1, replace=True, random_state=1)
# y_train2 = y_train.sample(frac=0.1, replace=True, random_state=1)
# X_test2 = X_test.sample(frac=0.1, replace=True, random_state=1)
# y_test2 = y_test.sample(frac=0.1, replace=True, random_state=1)
# print (X_train2.shape, y_train2.shape)
# print( X_test2.shape, y_test2.shape)

In [12]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X_train.values)
X_train = pd.DataFrame(x_scaled,columns=features)
x_scaled_test = min_max_scaler.fit_transform(X_test.values)
X_test = pd.DataFrame(x_scaled_test,columns=features)

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(100000, 64), # since it doesn't consider "words," the embedding doesn't really matter
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    # tf.keras.layers.Dense(1, activation='softmax') # loss too big
])

In [14]:
model.compile(loss='binary_crossentropy',
              # optimizer='sgd', # almost same
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train.values, y_train.values, epochs=3)

Train on 838860 samples
Epoch 1/3
838860/838860 [==============================] - 2614s 3ms/sample - loss: 0.6823 - accuracy: 0.5739
Epoch 2/3
838656/838860 [============================>.] - ETA: 0s - loss: 0.6822 - accuracy: 0.57

In [1]:
# test_loss, test_acc = model.evaluate(X_test2.values, y_test2.values, batch_size=64)

# print('Test Loss: {}'.format(test_loss))
# print('Test Accuracy: {}'.format(test_acc))

In [16]:
pred_class = model.predict(X_test.values[:])
print(pred_class)

[[0.46182874]
 [0.57929957]
 [0.57929957]
 ...
 [0.57929957]
 [0.57929957]
 [0.46182877]]


In [17]:
def myRound(x, r):
    if x>r/float(1000):
        return 1
    else:
        return 0

In [18]:
print(round(pred_class[2][0]))
print(round(pred_class[1][0]))

1.0
1.0


In [19]:
predictions = [int(round(x[0])) for x in pred_class]
true_class = list(y_test)

In [20]:
print(true_class[:10], len(true_class))
print(predictions[:10], len(predictions))

[0, 1, 1, 0, 1, 0, 1, 1, 1, 0] 209715
[0, 1, 1, 1, 1, 0, 1, 1, 1, 0] 209715


In [21]:
compdf = pd.DataFrame({'pred_class':predictions, 'true_class':true_class})
compdf = compdf.sort_values('pred_class', ascending=False)
print(str(compdf.head(10)))

        pred_class  true_class
104857           1           1
170280           1           1
170268           1           1
93496            1           1
93495            1           1
93494            1           1
170270           1           1
170273           1           1
93489            1           1
93488            1           1


In [22]:
predictions = list(compdf['pred_class'].apply(myRound, r=225))
true_class = list(compdf['true_class'])

In [23]:
print(predictions[:10])
print(true_class[:10])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [24]:
from sklearn.metrics import confusion_matrix

In [25]:
confm = confusion_matrix(true_class, predictions)
confm

array([[ 88450,    947],
       [     0, 120318]], dtype=int64)

In [26]:
tn, fp, fn, tp = confm.ravel()
(tn + tp)/(tn+tp+fn+fp)

0.9954843478053549

In [28]:
df2 = pd.read_csv('CustomDatasets/RegularHulkAttack.csv')
df2.head()

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.0.12-23.59.158.215-63949-443-6,23.59.158.215,443,192.168.0.12,63949,6,6/12/2019 9:20,1,1,1,...,0,0.000000e+00,0.000000e+00,0.0,0,0.000000e+00,0.000000e+00,0.0,0.0,Benign
1,192.168.0.12-192.168.0.15-0-0-0,192.168.0.15,0,192.168.0.12,0,0,6/12/2019 9:19,119471947,119,1,...,0,0.000000e+00,0.000000e+00,0.0,0,0.000000e+00,0.000000e+00,0.0,0.0,ModifiedHuLK
2,8.0.6.4-8.6.0.1-0-0-0,8.6.0.1,0,8.0.6.4,0,0,6/12/2019 9:19,115023918,23,1,...,0,4.249888e+06,6.028923e+06,15500000.0,126,6.886507e+06,3.949426e+06,19000000.0,5000053.0,Benign
3,192.168.0.1-239.255.255.250-51772-1900-17,192.168.0.1,51772,239.255.255.250,1900,17,6/12/2019 9:20,106023344,351,1,...,0,2.730851e+05,4.610152e+03,278251.0,266549,1.480000e+07,4.878160e+05,15700000.0,14100000.0,Benign
4,239.255.255.250-20.20.20.1-1900-33157-17,20.20.20.1,33157,239.255.255.250,1900,17,6/12/2019 9:20,106014555,351,1,...,0,2.647474e+05,4.230452e+03,270305.0,258567,1.480000e+07,4.878088e+05,15800000.0,14100000.0,Benign


In [4]:
features = ['Timestamp', 'Fwd Pkt Len Std', 'Fwd Pkt Len Mean',
       'Fwd Pkt Len Max', 'Fwd Seg Size Avg', 'Pkt Len Std', 'Flow IAT Std',
       'Bwd Pkt Len Std', 'Bwd Seg Size Avg', 'Pkt Size Avg',
       'Subflow Fwd Byts']

In [29]:
df2['Target']=df2['Label'].apply(targetify)

In [30]:
X2 = df2[features]
X2[features] = X2[features].apply(pd.to_numeric, errors='coerce', axis=1)

C:\Users\young\AppData\Roaming\Python\Python36\site-packages\pandas\core\frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [31]:
y2 = df2['Target']

In [32]:
X2 = X2.fillna(0)

In [33]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X2, y2, test_size=0.2)
print (X_train3.shape, y_train3.shape)
print( X_test3.shape, y_test3.shape)

(6966, 11) (6966,)
(1742, 11) (1742,)


In [11]:
# # selecting 1% of random rows for better running time

# X_train2 = X_train.sample(frac=0.1, replace=True, random_state=1)
# y_train2 = y_train.sample(frac=0.1, replace=True, random_state=1)
# X_test2 = X_test.sample(frac=0.1, replace=True, random_state=1)
# y_test2 = y_test.sample(frac=0.1, replace=True, random_state=1)
# print (X_train2.shape, y_train2.shape)
# print( X_test2.shape, y_test2.shape)

(83886, 11) (83886,)
(20972, 11) (20972,)


In [34]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled3 = min_max_scaler.fit_transform(X_train3.values)
X_train3 = pd.DataFrame(x_scaled3,columns=features)
x_scaled_test3 = min_max_scaler.fit_transform(X_test3.values)
X_test3 = pd.DataFrame(x_scaled_test3,columns=features)

In [35]:
pred_class3 = model.predict(X_test3.values[:])
print(pred_class3)

[[0.9943973]
 [0.9943973]
 [0.9943973]
 ...
 [0.9943973]
 [0.9943973]
 [0.9943973]]


In [36]:
print(round(pred_class3[2][0]))
print(round(pred_class3[1][0]))

1.0
1.0


In [37]:
predictions3 = [int(round(x[0])) for x in pred_class3]
true_class3 = list(y_test3)

In [38]:
print(true_class3[:10], len(true_class3))
print(predictions3[:10], len(predictions3))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1742
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1742


In [39]:
compdf3 = pd.DataFrame({'pred_class3':predictions3, 'true_class3':true_class3})
compdf3 = compdf3.sort_values('pred_class3', ascending=False)
print(str(compdf3.head(10)))

      pred_class3  true_class3
0               1            1
1158            1            1
1169            1            1
1168            1            1
1167            1            1
1166            1            1
1165            1            1
1164            1            1
1163            1            1
1162            1            1


In [41]:
predictions3 = list(compdf3['pred_class3'].apply(myRound, r=225))
true_class3 = list(compdf3['true_class3'])

In [42]:
print(predictions3[:10])
print(true_class3[:10])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [25]:
from sklearn.metrics import confusion_matrix

In [43]:
confm3 = confusion_matrix(true_class3, predictions3)
confm3

array([[   1,   44],
       [   0, 1697]], dtype=int64)

In [44]:
tn3, fp3, fn3, tp3 = confm3.ravel()
(tn3 + tp3)/(tn3+tp3+fn3+fp3)

0.9747416762342136

In [48]:
X_train5, X_test5, y_train5, y_test5 = train_test_split(X2, y2, test_size=0.99)
print (X_train5.shape, y_train5.shape)
print( X_test5.shape, y_test5.shape)
print( X2.shape, y2.shape)

(87, 11) (87,)
(8621, 11) (8621,)
(8708, 11) (8708,)


In [51]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled_test5 = min_max_scaler.fit_transform(X2.values)
X3 = pd.DataFrame(x_scaled_test5,columns=features)
print(X3.shape, y2.shape)

(8708, 11) (8708,)


In [52]:
pred_class5 = model.predict(X3.values[:])
print(pred_class5)

[[0.9943973]
 [0.9943973]
 [0.9943973]
 ...
 [0.9943973]
 [0.9943973]
 [0.9943973]]


In [53]:
predictions5 = [int(round(x[0])) for x in pred_class5]
true_class5 = list(y2)

In [54]:
compdf5 = pd.DataFrame({'pred_class5':predictions5, 'true_class5':true_class5})
compdf5 = compdf5.sort_values('pred_class5', ascending=False)
print(str(compdf5.head(10)))

      pred_class5  true_class5
0               1            0
5807            1            1
5801            1            1
5802            1            1
5803            1            1
5804            1            1
5805            1            1
5806            1            1
5808            1            1
4352            1            1


In [55]:
predictions5 = list(compdf5['pred_class5'].apply(myRound, r=225))
true_class5 = list(compdf5['true_class5'])

In [56]:
confm5 = confusion_matrix(true_class5, predictions5)
confm5

array([[   9,  249],
       [   0, 8450]], dtype=int64)

In [57]:
tn5, fp5, fn5, tp5 = confm5.ravel()
(tn5 + tp5)/(tn5+tp5+fn5+fp5)

0.97140560404226